## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [3]:
#mean = 'PRCP'
mean = 'TOBS'

In [4]:
data_dir = "../../Data/Weather"
from pickle import load
with open(data_dir+'/'+mean+'_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl

c_filename="stations.pkl.gz"
u_filename="stations.pkl"

#unzip
!gunzip -c $data_dir/$c_filename > $data_dir/$u_filename

with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


latitude  longitude  elevation state                   name  \
ACW00011604   17.1167   -61.7833       10.1   NaN  ST JOHNS COOLIDGE FLD   
ACW00011647   17.1333   -61.7833       19.2   NaN               ST JOHNS   
AE000041196   25.3330    55.5170       34.0   NaN    SHARJAH INTER. AIRP   
AF000040930   35.3170    69.0170     3366.0   NaN           NORTH-SALANG   
AG000060390   36.7167     3.2500       24.0   NaN     ALGER-DAR EL BEIDA   

            GSNFLAG HCNFLAG    WMOID  
ACW00011604     NaN     NaN      NaN  
ACW00011647     NaN     NaN      NaN  
AE000041196     GSN     NaN  41196.0  
AF000040930     GSN     NaN  40930.0  
AG000060390     GSN     NaN  60390.0

In [5]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

0         1         2         3
0 -0.281820 -0.160174  0.050505 -0.024916
1 -0.215372  0.145648  0.230682 -0.085874
2 -0.083278  0.080262 -0.537689  0.350890
3  0.015830  0.043151  0.141461 -0.083158
4 -0.337709  0.186580 -0.168956  0.229560

In [6]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00012813', u'USC00011084', u'USC00013035', u'USC00012675', u'USC00011080']


0         1         2         3
station                                            
USC00012813 -0.281820 -0.160174  0.050505 -0.024916
USC00011084 -0.215372  0.145648  0.230682 -0.085874
USC00013035 -0.083278  0.080262 -0.537689  0.350890
USC00012675  0.015830  0.043151  0.141461 -0.083158
USC00011080 -0.337709  0.186580 -0.168956  0.229560

In [7]:
table=Eig.join(Stations,how='left')

In [8]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

name  latitude  longitude  elevation         0  \
station                                                                 
USC00012813   FAIRHOPE 2 NE   30.5467   -87.8808        7.0 -0.281820   
USC00011084   BREWTON 3 SSE   31.0583   -87.0550       25.9 -0.215372   
USC00013035       FT MORGAN   30.2333   -88.0167        3.0 -0.083278   
USC00012675  ENTERPRISE 4 W   31.2975   -85.8997      102.7  0.015830   

                    1         2         3  
station                                    
USC00012813 -0.160174  0.050505 -0.024916  
USC00011084  0.145648  0.230682 -0.085874  
USC00013035  0.080262 -0.537689  0.350890  
USC00012675  0.043151  0.141461 -0.083158

In [9]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [10]:
type(table)

pandas.core.frame.DataFrame

In [11]:
max_lat = float( table['latitude'].max() )
min_lat = float( table['latitude'].min() )

max_long = float( table['longitude'].max() )
min_long = float( table['longitude'].min() )

print 'box boundry: ', min_lat,max_lat,min_long,max_long

box boundry:  30.1667 31.5253 -88.4833 -85.5833


## Map

In [12]:
box = min_lat,max_lat,min_long,max_long
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   

m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [13]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined